In [1]:
#Diffusion Sigma analysis
#Get final R position of a drift line (no diffusion)
#   - Compare to 100 diffusion lines (Use a blue histogram and a black x)
#   - Fit a gaussian to the histogram - extract mu and sigma
#   - Compare Gaussian to mu and sigma to final drift position and sqrt(4*D_T*DriftTime)
#   - Repeat for 9 positions (0,2) --> (72.5, 2)

In [2]:

#INITIALIZION
#Kernel PyROOT
import os
#os.sys.path.append('/usr/common/software/rootpy')
#os.sys.path.insert(0,'/usr/common/software/uproot')
#import root_numpy
#import root_numpy as root_np
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
#matplotlib.use('GTK3Agg')
import ROOT
import pandas as pd

import sys
from matplotlib.colors import LogNorm
import pylab

import glob
import uproot

import scipy
from scipy.optimize import curve_fit
from scipy.stats import norm

%pylab inline
#pylab.rcParams['figure.figsize'] = (10.0, 8.0)

%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (10.0, 8.0)

print("Starting notebook....")

#Function to get directory list
def Get_folders_in_dir(d):
    [os.path.join(d, o) for o in os.listdir(d) 
                        if os.path.isdir(os.path.join(d,o))]
    return os.listdir(d)

#For individual subdir get all files
def Get_All_files(d):
    file_list=[]
    for file in os.listdir(d):
        if file.endswith(".txt"):
            #print(os.path.join(file))
            file_list.append(d+os.path.join(file))
    return file_list

#Get the i^th entry from a file
def Get_i_line_from_file(filename, i):
    file = open(filename, 'r')
    lines = file.readlines()
    a = lines[i].split('\t\t')
    DriftTime_us= float(a[2])
    r_cm = float(a[3])
    z_cm = float(a[4])
    return r_cm, z_cm, DriftTime_us


#Get list of final R and driftline positions
def Get_list_of_final_R_DT(file_list):
    r_list = []
    DT_list = []
    for i in range(0,len(file_list)):
        j=-1
        r,z,t= Get_i_line_from_file(file_list[i],j)
        r_list.append(r)
        DT_list.append(t)
        #CHECK to make sure drift time makes sense
        if t > 800:
            print str(r)+"\t"+str(z)+"\t"+str(t)
    return r_list, DT_list

#Function to condense diffusion lines into single file
def MakeCondensedDiffusionFile(directory):
    #Get original r position of sim
    print "directory: "+str(directory)
    r_origin = directory.split("/")[-2].split("r")[1]
    print "r_origin = "+str(r_origin)
    #For individual subdir get all files
    file_list = Get_All_files(directory)
    #print "1st file = "+str(file_list[0])
    #For one diffusion line get last entry which makes sense        
    r_list, DT_list = Get_list_of_final_R_DT(file_list)

    #print "len(r_list) = "+str(len(r_list))
    #print "len(DT_list) = "+str(len(DT_list))

    print "len(r_list) = "+str(len(r_list))+" \tlen(DT_list) = "+str(len(DT_list))
    print "\n\t... lists retreived!"

    #Condense r_list, and, DT_list into a single file
    #d_4 = "/data/rossiter/lz/EDM_DiffusionAnalysis_2005_2/Condensed/"
    d_4 = "/data/rossiter/lz/EDM_DiffusionAnalysis_2007/Condensed/"
    output_filename = d_4+"r"+r_origin+'.txt'
    print "output_filename = "+str(output_filename)

    f=open(output_filename, "w")
    f.write("#r_cm\tDT_us\n")
    for i in range(0, len(r_list)):
        f.write(str(r_list[i])+"\t"+str(DT_list[i])+"\n")

    f.close

    print "\n\t ...r"+r_origin+'.txt written!'
    return 0



#Gaussian Function
def Gaussian(x, mu, sigma):
    A= 1/(sigma*pow(2*pi,.5))
    B= -.5*pow((x-mu)/sigma, 2.)
    return A*np.exp(B)
vGaussian = np.vectorize(Gaussian)

#Function to turn file into arrays
def Get_drift_line_arrays(filename):
    file = open(filename, 'r')
    lines = file.readlines()

    #print len(lines)
    #print lines[2]

    rIN_cm = float(lines[10].split('\t\t')[0])
    zIN_cm = float(lines[10].split('\t\t')[1])

    DriftTime_us_list = []
    r_cm_list = []
    z_cm_list = []

    DriftTime_us_list.append(0.0)
    r_cm_list.append(rIN_cm)
    z_cm_list.append(zIN_cm)

    for i in range(0,len(lines)):
        if lines[i].startswith('#'):
            continue
        elif lines[i].startswith('*'):
            print lines[i]
        elif lines[i].startswith('0') or lines[i].startswith('1') or lines[i].startswith('2')\
        or lines[i].startswith('3') or lines[i].startswith('4') or lines[i].startswith('5')\
        or lines[i].startswith('6') or lines[i].startswith('7') or lines[i].startswith('8')\
        or lines[i].startswith('9'):
            a = lines[i].split('\t\t')
            #print a
            DriftTime_us_list.append(float(a[2]))
            r_cm_list.append(float(a[3]))
            z_cm_list.append(float(a[4]))
        else:
            continue
    return np.asarray(r_cm_list), np.asarray(z_cm_list), np.asarray(DriftTime_us_list)



def Get_file_length(filename):
    file = open(filename, 'r')
    lines = file.readlines()
    return len(lines)




from scipy.stats import norm

def Get_Zeroed_R_list(filename, data_dir):
    print data_dir
    file_list=[]
    for file in os.listdir(data_dir):
        if file.endswith(".txt"):
            print(os.path.join(file))
            file_list.append(data_dir+os.path.join(file))
           
    r_list = []
    z_list = []
    t_list = []
    j=-1
    #print j
    for f in file_list[:-1]:
        r,z,t= Get_i_line_from_file(f,j)
        r_list.append(r)
        z_list.append(z)
        t_list.append(t)
    r,z,t=Get_i_line_from_file(filename,pop)
    print "\t"+str(filename)
    zeroed_r_array = np.asarray(r_list)-r
    
    fit_mu, fit_sigma = norm.fit(zeroed_r_array)
    
    #Inspect initial histogram
    num_bins =  10
    #(full_n, full_bins, full_patches) = plt.hist(Full_Reduced_R_list, bins=num_bins)
    figure(1)
    (n_R, bins, patches) = plt.hist(zeroed_r_array, bins=num_bins)

    #Get bin mids
    print bins.shape
    print bins

    R_bin_mids_list = []
    for i in range(1, len(bins)):
        R_bin_mids_list.append( (bins[i] + bins[i-1])/2 )

    this_half_bid_width = .5*(R_bin_mids_list[1] - R_bin_mids_list[0])

    #print bin_mids_list[-1]
    #print bin_mids_list[-1]+this_half_bid_width
    #print bin_mids_list[-1]-this_half_bid_width


    #Remove bins with n<threshold, & remove top bin
    threshold=0


    counter = 0
    new_bin_mids_list = []
    new_n_list = []
    for i in range(0, len(n_R)):
        if n_R[i] < threshold:# or n_R[i] > 15000:
            #print( str(i)+": ("+str(bin_mids_list[i])+", "+str(n[i])+")" )
            counter +=1
        else:
            new_bin_mids_list.append(R_bin_mids_list[i])
            new_n_list.append(n_R[i])

    #Inspect elements with n>10
    #figure(2)
    #plt.plot(R_bin_mids_list, n_R, ".")
    #plt.plot(new_bin_mids_list,  new_n_list, 'x')
    
    return zeroed_r_array, fit_mu, fit_sigma, new_bin_mids_list, new_n_list
    


print "\n\t...Done!"

Welcome to JupyROOT 6.16/00
Populating the interactive namespace from numpy and matplotlib
Starting notebook....

	...Done!


/cvmfs/sft.cern.ch/lcg/views/LCG_95/x86_64-centos7-gcc8-opt/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['pylab', 'norm']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [3]:
#Get list of directorys for diffusion lines
d_1 = "/data/rossiter/lz/EDM_DiffusionAnalysis_2005/"
#d_2 = "/data/rossiter/lz/EDM_DiffusionAnalysis_2005_2/"
d_2 = "/data/rossiter/lz/EDM_DiffusionAnalysis_2007/"
dir_list = []

temp_dir_list = Get_folders_in_dir(d_2)

for d in temp_dir_list:
    if d.startswith('r'):
        #print d
        dir_list.append(d_2+d+'/')

#temp_dir_list = Get_folders_in_dir(d_1)
#for d in temp_dir_list:
#    if d=='r0':# or d =='r70':
#        #print d
#        dir_list.append(d_1+d+'/')

print dir_list[0]

print "\n\t...Done!"

/data/rossiter/lz/EDM_DiffusionAnalysis_2007/r00/

	...Done!


In [4]:
for ii in range(0,len(dir_list)):
    #r_origin = dir_list[ii].split("/")[-2].split("r")[1]
    #print r_origin
    temp = MakeCondensedDiffusionFile(dir_list[ii])
    
print "\n\t...Done!"

directory: /data/rossiter/lz/EDM_DiffusionAnalysis_2007/r00/
r_origin = 00
len(r_list) = 1000 	len(DT_list) = 1000

	... lists retreived!
output_filename = /data/rossiter/lz/EDM_DiffusionAnalysis_2007/Condensed/r00.txt

	 ...r00.txt written!
directory: /data/rossiter/lz/EDM_DiffusionAnalysis_2007/r08.75/
r_origin = 08.75
len(r_list) = 1000 	len(DT_list) = 1000

	... lists retreived!
output_filename = /data/rossiter/lz/EDM_DiffusionAnalysis_2007/Condensed/r08.75.txt

	 ...r08.75.txt written!
directory: /data/rossiter/lz/EDM_DiffusionAnalysis_2007/r52.5/
r_origin = 52.5
len(r_list) = 1000 	len(DT_list) = 1000

	... lists retreived!
output_filename = /data/rossiter/lz/EDM_DiffusionAnalysis_2007/Condensed/r52.5.txt

	 ...r52.5.txt written!
directory: /data/rossiter/lz/EDM_DiffusionAnalysis_2007/r43.75/
r_origin = 43.75
len(r_list) = 1000 	len(DT_list) = 1000

	... lists retreived!
output_filename = /data/rossiter/lz/EDM_DiffusionAnalysis_2007/Condensed/r43.75.txt

	 ...r43.75.txt written!
